In [2]:
import pandas as pd
import torch
from torch import nn
import gensim
from gensim.models import KeyedVectors
import sklearn
import torchmetrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df3 = pd.read_csv('main_dataset.csv',  encoding='UTF-8')
df3.head()

,id,text,label
0,1,на работе был полный пиддес :| и так каждое за...,0.0
1,2,"Коллеги сидят рубятся в Urban terror, а я из-з...",1.0
2,3,@elina_4post как говорят обещаного три года жд...,0.0
3,4,"Желаю хорошего полёта и удачной посадки,я буду...",0.0
4,5,"Обновил за каким-то лешим surf, теперь не рабо...",0.0


In [4]:
df3.tail()

,id,text,label
15870,15871,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
15871,15872,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
15872,15873,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
15873,15874,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0
15874,15875,До сих пор пересматриваю его видео. Орамбо кст...,0.0


In [5]:
df3.isna().sum()

id       0
text     0
label    0
dtype: int64

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
print(df3['text'].apply(type).value_counts())

<class 'str'>    15875
Name: text, dtype: int64


In [8]:
!pip install stop-words


In [9]:
import stop_words
#stop_words_russian = set(stopwords.words('russian'))
stop_words_russian = stop_words.get_stop_words('russian')

In [10]:
# Convert non-string values to empty string
df3['text'] = df3['text'].apply(lambda x: str(x) if type(x) != str else x)

# Remove stop words from the text column
df3['text'] = df3['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_russian]))

In [11]:
df3.head(15)

,id,text,label
0,1,"работе полный пиддес :| закрытие месяца, свихн...",0.0
1,2,"Коллеги сидят рубятся Urban terror, из-за долб...",1.0
2,3,@elina_4post говорят обещаного ждут...((,0.0
3,4,"Желаю хорошего полёта удачной посадки,я сильно...",0.0
4,5,"Обновил каким-то лешим surf, работает простопл...",0.0
5,6,"Котёнка вчера носик разбила, плакала расстраив...",0.0
6,7,"@juliamayko @O_nika55 @and_Possum Зашли, затих...",0.0
7,8,вообще болею - выздоравливаю :(,0.0
8,9,микрофраза :( учимся срать кирпичами режиме &a...,1.0
9,10,"хочу помириться , сука гордая сделаю! (((",1.0


In [12]:
import re
def remove_urls(text):
    return re.sub(r'http\S+', '', text)
# Apply the function to the 'text' column of the DataFrame
df3['text'] = df3['text'].apply(remove_urls)

In [13]:
df3.tail(20)

,id,text,label
15855,15856,"ТАСС, 21 марта. Премьер-министр Новой Зеландии...",0.0
15856,15857,"вой подсоса гомофорсера Что, говно, неприятно ...",1.0
15857,15858,Шлюха. парад шлюх.,1.0
15858,15859,дебил-куколд Значение знаешь?,1.0
15859,15860,"хуею сосачеры, большиство прыщавые жиртресты р...",1.0
15860,15861,Что-то появившееся небе пугает туристов расска...,1.0
15861,15862,Моча анимаче решила окончательно убить реакшен...,1.0
15862,15863,65 мало? гражданина рейтинг 90 3 дня записи ви...,0.0
15863,15864,"зря, вас, хохлов, свиньями кличут. грязные жив...",1.0
15864,15865,прямо сразу ватан бросился своему кредитному н...,1.0


In [14]:
import string

In [15]:
# Define a function to remove punctuation and extra spaces from a text string
def remove_punct_and_extra_space(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra space
    text = re.sub(' +', ' ', text)
    return text

# Apply the function to the 'text' column of the DataFrame
df3['text'] = df3['text'].apply(remove_punct_and_extra_space)

In [16]:
df3.tail(15)

,id,text,label
15860,15861,Чтото появившееся небе пугает туристов рассказ...,1.0
15861,15862,Моча анимаче решила окончательно убить реакшен...,1.0
15862,15863,65 мало гражданина рейтинг 90 3 дня записи видео,0.0
15863,15864,зря вас хохлов свиньями кличут грязные животны...,1.0
15864,15865,прямо сразу ватан бросился своему кредитному н...,1.0
15865,15866,ЭЙ МИХАЛЫЧ ПОЖАРКА ПИЩИТ ХУЙ ПУЩАЙ ПИЩИТ СМОРИ...,1.0
15866,15867,Пусть евровидение стримит Долбоеб чтоли послед...,1.0
15867,15868,ЖЕНЩИНА ВЕНЕЦ ТВОРЕНИЯ помните ваньки стоите ж...,1.0
15868,15869,авиакомпании вместе специалистами гугл карт см...,1.0
15869,15870,запад прошел хуйню пару сотен назад Большинств...,0.0


In [17]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [18]:
#df3['text'] = df3['text'].apply(lambda x: nltk.word_tokenize(x))

In [19]:
df3.head()

,id,text,label
0,1,работе полный пиддес закрытие месяца свихнусь D,0.0
1,2,Коллеги сидят рубятся Urban terror изза долбан...,1.0
2,3,elina4post говорят обещаного ждут,0.0
3,4,Желаю хорошего полёта удачной посадкия сильно ...,0.0
4,5,Обновил какимто лешим surf работает простоплеер,0.0


In [20]:
!pip install pymorphy2

In [21]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
from nltk.corpus import stopwords

In [22]:
import re
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) >= 1:
        return tokens
    return None
df3['text'] = df3.text.apply(lemmatize)
df3.iloc[19:24]

,id,text,label
19,20,"[какойтый, нехороший, телефон, наушник, опозна...",0.0
20,21,"[думаюна, мой, век, почта, россия, подняться, ...",0.0
21,22,"[один, напоминать, дом, путин, делать, реалити...",0.0
22,23,"[ебааааа, модуль, региональный, экономика]",1.0
23,24,"[оооууа, анфолловить, чувак]",0.0


In [23]:
df3.to_csv('df3_after_preprocessing.csv', index=False)

In [24]:
after = pd.read_csv('df3_after_preprocessing.csv',  encoding='UTF-8')
after.head()

,id,text,label
0,1,"['работа', 'полный', 'пиддес', 'закрытие', 'ме...",0.0
1,2,"['коллега', 'сидеть', 'рубиться', 'изз', 'долб...",1.0
2,3,"['говорить', 'обещаной', 'ждать']",0.0
3,4,"['желать', 'хороший', 'полёт', 'удачный', 'пос...",0.0
4,5,"['обновить', 'какимтый', 'леший', 'работать', ...",0.0


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
# Удаление строк с пропущенными значениями в столбце 'text'
after = after.dropna(subset=['text'])

# Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование текстов в матрицу TF-IDF
tfidf_matrix = vectorizer.fit_transform(after['text'])

# Получение словаря признаков (слов)
features = vectorizer.vocabulary_

# Вывод матрицы TF-IDF
print(tfidf_matrix.toarray())


print('\n')
# Вывод списка признаков
print(features)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


{'работа': 24965, 'полный': 22022, 'пиддес': 20556, 'закрытие': 8709, 'месяц': 14378, 'свихнуться': 27175, 'коллега': 11576, 'сидеть': 27556, 'рубиться': 26511, 'изз': 9821, 'долбать': 7089, 'винд': 3293, 'мочь': 15061, 'говорить': 5409, 'обещаной': 17512, 'ждать': 8003, 'желать': 8021, 'хороший': 33351, 'полёт': 22138, 'удачный': 31570, 'посадкия': 22519, 'сильно': 27585, 'скучать': 27948, 'обновить': 17648, 'какимтый': 10628, 'леший': 13143, 'работать': 24971, 'простоплеер': 24387, 'котёнок': 12207, 'вчера': 4249, 'носик': 17279, 'разбить': 25089, 'плакать': 20858, 'расстраиваться': 25668, 'заслать': 9100, 'затихариться': 9183, 'прямо': 24632, 'физически': 32568, 'страдать': 29489, 'молчать': 14887, 'вообще': 3782, 'болеть': 2075, 'выздоравливать': 4407, 'микрофраза': 14508, 'учиться': 32306, 'срать': 29090, 'кирпич': 11230, 'ре

In [27]:
print(type(features))

<class 'dict'>


In [28]:
import torch
from sklearn.feature_extraction.text import TfidfVectorizer

# Удаление строк с пропущенными значениями в столбце 'text'
after = after.dropna(subset=['text'])

# Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование текстов в матрицу TF-IDF
tfidf_matrix = vectorizer.fit_transform(after['text'])

# Преобразование матрицы TF-IDF в тензор PyTorch
tfidf_tensor = torch.tensor(tfidf_matrix.toarray(), dtype=torch.float32)

# Вывод тензора TF-IDF
print(tfidf_tensor)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [29]:
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

#Удаление строк с пропущенными значениями в столбце 'text'
after = after.dropna(subset=['text'])

#Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

#Преобразование текстов в матрицу TF-IDF
tfidf_matrix = vectorizer.fit_transform(after['text'])

#Преобразование разреженной матрицы в формат CSR
tfidf_csr = csr_matrix(tfidf_matrix)

# Преобразование формата разреженной матрицы в формат COO
tfidf_coo = tfidf_csr.tocoo().astype(np.float32)

# Преобразование матрицы TF-IDF в разреженный тензор PyTorch
tfidf_tensor = torch.sparse.FloatTensor(torch.LongTensor([tfidf_coo.row.tolist(), tfidf_coo.col.tolist()]),
                                        torch.FloatTensor(tfidf_coo.data))

# Вывод тензора TF-IDF
print(tfidf_tensor)

tensor(indices=tensor([[  933,  1867, 10723,  ...,   136,   891,   623],
                       [    0,     0,     0,  ..., 35478, 35479, 35480]]),
       values=tensor([0.2858, 0.1991, 0.3850,  ..., 0.4826, 0.5030, 0.2085]),
       size=(15858, 35481), nnz=204570, layout=torch.sparse_coo)


In [30]:
x_train=tfidf_tensor

In [31]:
x_train.size()

torch.Size([15858, 35481])

In [32]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
#         out=torch.round(out)
        return out

In [ ]:
if torch.cuda.is_available():
    dev=torch.device('cuda:0')
else: dev=torch.device('cpu:0')

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [33]:
optimizer = torch.optim.Adam(Mk1.parameters(), lr=0.001)
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy'])
for epoch in range(10000):
    optimizer.zero_grad()
    pred = Mk1.forward(x_train.float())
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf = pd.concat([diagdf, pd.DataFrame(data, index=[epoch])], ignore_index=True)
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

NameError: name 'Mk1' is not defined